# Universidad Autonoma de Aguascalientes
# Departamento: Ciencias de la Computación
# Materia: Machine y Deep Learning
# Profesor: Dr. Francisco Javier Luna Rosas
# Alumnos: 
# Enrique Vélez Durán
# Gabriel Melchor Campos
# Carlos Fernando Nájera Ruiz
# Cristián Israel Donato Flores
#### Semestre: Enero-Junio 2025
## Reconocimiento de Emociones con NNBP
## Este programa detecta emociones en tiempo real usando una red neuronal artificial, analiza rostros captados por la cámara y predice la emoción predominante. Incluye la graficación de la pérdida del entrenamiento, la predicción de emociones a partir de imágenes y la detección en tiempo real mostrando la emoción con su nivel de confianza.

## Importación Librerías

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from datetime import datetime

## Definición de la Clase EmotionNNBP
### Este bloque define la red neuronal y sus métodos principales

In [ ]:
class EmotionNNBP:
    def __init__(self):
        self.emotions = ['feliz', 'enojado', 'triste', 'sorprendido', 'neutral']
        self.data_path = 'dataset'
        self.model_path = 'emotion_model_nnbp.npz'
        self.input_size = 48 * 48
        self.hidden_size = 256
        self.output_size = len(self.emotions)
        self.learning_rate = 0.001
        self.epochs = 100
        self.batch_size = 32

        # Inicialización de pesos con Xavier/Glorot
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * np.sqrt(2.0 / self.input_size)
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * np.sqrt(2.0 / self.hidden_size)
        self.b2 = np.zeros((1, self.output_size))

        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def save_model(self):
        np.savez(self.model_path, W1=self.W1, b1=self.b1, W2=self.W2, b2=self.b2)

    def load_model(self):
        try:
            model_data = np.load(self.model_path)
            self.W1 = model_data['W1']
            self.b1 = model_data['b1']
            self.W2 = model_data['W2']
            self.b2 = model_data['b2']
            return True
        except Exception as e:
            print(f"Error al cargar el modelo: {e}")
            return False


## Creación del Dataset (Captura de Imágenes)
### Este bloque captura imágenes con la cámara en tiempo real. Se guardarán en carpetas organizadas por cada emoción

In [ ]:
    def create_dataset_structure():
        os.makedirs('dataset', exist_ok=True)
        emotions = ['feliz', 'enojado', 'triste', 'sorprendido', 'neutral']
        for emotion in emotions:
            os.makedirs(os.path.join('dataset', emotion), exist_ok=True)

    def capture_dataset():
        create_dataset_structure()
        cap = cv2.VideoCapture(0)

        emotions = ['feliz', 'enojado', 'triste', 'sorprendido', 'neutral']

        for emotion in emotions:
            count = 0
            input(f"Presiona Enter para capturar imágenes de {emotion}...")
            print(f"Capturando {emotion}... Presiona 'q' para salir.")

            while count < 140:
                ret, frame = cap.read()
                if not ret:
                    break

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)

                for (x, y, w, h) in faces:
                    face_roi = cv2.resize(gray[y:y+h, x:x+w], (48, 48))
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
                    file_path = os.path.join('dataset', emotion, f'{emotion}_{timestamp}.jpg')
                    cv2.imwrite(file_path, face_roi)
                    count += 1
                    print(f"Capturada {count}/140 imágenes de {emotion}")

            print(f"Finalizada captura de {emotion}")

        cap.release()
        cv2.destroyAllWindows()

    # Ejecutar solo cuando se desee capturar imágenes
    # capture_dataset()


## Cargar y Prepocesar Datos
### Se cargán las imágenes desde las carpetas y se dividen en conjunto de entrenamiento y prueba

In [ ]:
def load_data(self):
    """Carga y preprocesa las imágenes del dataset"""
    X = []
    y = []

    # Verificar si existe el directorio del dataset
    if not os.path.exists(self.data_path):
        raise FileNotFoundError(f"No se encontró el directorio del dataset: {self.data_path}")

    print("Cargando dataset...")
    for idx, emotion in enumerate(self.emotions):
        emotion_path = os.path.join(self.data_path, emotion)

        # Verificar si existe el directorio de la emoción
        if not os.path.exists(emotion_path):
            print(f"Advertencia: No se encontró el directorio para {emotion}")
            continue

        # Obtener lista de imágenes
        image_files = [f for f in os.listdir(emotion_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

        if not image_files:
            print(f"Advertencia: No se encontraron imágenes para {emotion}")
            continue

        print(f"Cargando imágenes de {emotion}: {len(image_files)} encontradas")

        for img_file in image_files:
            img_path = os.path.join(emotion_path, img_file)
            try:
                # Leer y preprocesar imagen
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Error al cargar imagen: {img_path}")
                    continue

                # Asegurar que la imagen sea 48x48
                img = cv2.resize(img, (48, 48))

                # Aplanar y normalizar la imagen
                img_flat = img.flatten() / 255.0

                X.append(img_flat)
                y.append(idx)

            except Exception as e:
                print(f"Error procesando {img_path}: {e}")
                continue

    if not X or not y:
        raise ValueError("No se pudieron cargar imágenes. Asegúrate de capturar el dataset primero.")

    X = np.array(X)
    y = np.array(y)

    # Convertir etiquetas a one-hot encoding
    y = to_categorical(y, num_classes=len(self.emotions))

    print(f"\nDataset cargado exitosamente:")
    print(f"Tamaño del dataset: {len(X)} imágenes")
    print(f"Dimensiones de entrada: {X.shape}")
    print(f"Dimensiones de salida: {y.shape}")

    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    print(f"\nConjunto de entrenamiento: {X_train.shape[0]} imágenes")
    print(f"Conjunto de prueba: {X_test.shape[0]} imágenes")

    return X_train, X_test, y_train, y_test


## Cargado y Guardado del Modelo
### Carga el entrenamiento y guarda el modelo

In [ ]:
def save_model(self):
    """Guarda el modelo en formato .npz"""
    np.savez(self.model_path.replace('.npy', '.npz'),
            W1=self.W1, b1=self.b1, W2=self.W2, b2=self.b2)

def load_model(self):
    """Carga el modelo desde formato .npz"""
    try:
        model_data = np.load(self.model_path.replace('.npy', '.npz'))
        self.W1 = model_data['W1']
        self.b1 = model_data['b1']
        self.W2 = model_data['W2']
        self.b2 = model_data['b2']
        return True
    except Exception as e:
        print(f"Error al cargar el modelo: {e}")
        return False

## Entrenamiento del Modelo
### Entrena la red neuronal con las imágenes capturadas

In [ ]:
def train(self):
    try:
        X_train, X_test, y_train, y_test = self.load_data()
    except Exception as e:
        print(f"Error al cargar los datos: {e}")
        return

    print("Iniciando entrenamiento...")
    losses = []
    best_loss = float('inf')
    patience = 5
    patience_counter = 0

    for epoch in range(self.epochs):
        # Mini-batch training
        indices = np.random.permutation(len(X_train))
        total_loss = 0

        for i in range(0, len(X_train), self.batch_size):
            batch_indices = indices[i:i + self.batch_size]
            X_batch = X_train[batch_indices]
            y_batch = y_train[batch_indices]

            # Forward pass
            Z1 = np.dot(X_batch, self.W1) + self.b1
            A1 = self.sigmoid(Z1)
            Z2 = np.dot(A1, self.W2) + self.b2
            A2 = self.sigmoid(Z2)

            # Backward pass
            loss = np.mean((y_batch - A2) ** 2)
            total_loss += loss

            error_output = (y_batch - A2) * self.sigmoid_derivative(A2)
            error_hidden = np.dot(error_output, self.W2.T) * self.sigmoid_derivative(A1)

            # Update weights and biases
            self.W2 += self.learning_rate * np.dot(A1.T, error_output)
            self.b2 += self.learning_rate * np.sum(error_output, axis=0, keepdims=True)
            self.W1 += self.learning_rate * np.dot(X_batch.T, error_hidden)
            self.b1 += self.learning_rate * np.sum(error_hidden, axis=0, keepdims=True)

        avg_loss = total_loss / (len(X_train) / self.batch_size)
        losses.append(avg_loss)

        if epoch % 5 == 0:
            print(f'Época {epoch}/{self.epochs} - Pérdida: {avg_loss:.4f}')

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
            # Guardar mejor modelo
            self.save_model()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping activado")
                break

    self.plot_training_loss(losses)
    print("Entrenamiento completado")

## Gráfica la perdida del entrenamiento

In [ ]:
def plot_training_loss(self, losses):
    plt.figure(figsize=(10, 6))
    plt.plot(losses)
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.title('Evolución de la pérdida en el entrenamiento')
    plt.grid(True)
    plt.savefig('training_loss_nnbp.png')
    plt.close()


## Predicción de emociones

In [ ]:
def predict_emotion(self, face_roi):
    face_roi = face_roi.flatten() / 255.0  
    Z1 = np.dot(face_roi, self.W1) + self.b1  
    A1 = self.sigmoid(Z1)
    Z2 = np.dot(A1, self.W2) + self.b2  
    A2 = self.sigmoid(Z2)
    return np.argmax(A2), np.max(A2)  


## Detección de Emociones en Tiempo Real

In [ ]:
def detect_emotion(self):
    if not self.load_model():
        print("Por favor, entrena el modelo primero.")
        return

    cap = cv2.VideoCapture(0)  # Inicia la cámara
    print("Presiona 'q' para salir")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convierte a escala de grises
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)  # Detecta rostros

        for (x, y, w, h) in faces:
            face_roi = cv2.resize(gray[y:y+h, x:x+w], (48, 48))  # Ajusta el tamaño
            emotion_idx, confidence = self.predict_emotion(face_roi)  # Predice emoción
            emotion = self.emotions[emotion_idx]

            # Dibujar rectángulo y etiqueta
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            label = f"{emotion} ({confidence:.1%})"
            cv2.putText(frame, label, (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                        (0, 255, 0), 2)

        cv2.imshow('Detector de Emociones', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con 'q'
            break

    cap.release()
    cv2.destroyAllWindows()


## Menú Principal

In [ ]:
def main():
    modelo = EmotionNNBP()
    while True:
        print("\nSistema de Reconocimiento de Emociones NNBP")
        print("============================================")
        print("0. Salir")
        print("1. Capturar dataset")
        print("2. Entrenar modelo")
        print("3. Detectar emociones en tiempo real")

        try:
            opcion = int(input("\nElige una opción (0-3): "))

            if opcion == 0:
                print("¡Hasta luego!")
                break
            elif opcion == 1:
                modelo.capture_dataset()
            elif opcion == 2:
                modelo.train()
            elif opcion == 3:
                if not os.path.exists(modelo.model_path.replace('.npy', '.npz')):
                    print("Error: No se encontró el modelo entrenado.")
                    print("Por favor, entrena el modelo primero (opción 2).")
                else:
                    modelo.detect_emotion()
            else:
                print("Opción no válida. Por favor, elige una opción entre 0 y 3.")
        except ValueError:
            print("Por favor, ingresa un número válido.")
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    main()